
#  How compilers translate your program


Enter your name and student ID.

 * Name:
 * Student ID:



<a name="intro"> </a>
# 1. Introduction
* You look at assembly code generated by the compiler to get a good sense of how source programs are compiled into machine (assembly) code



# 2. Let the compiler generate assembly code
* when you compile a program or build a project, a compiler generally compiles it all the way into an executable
* that is, machine (or assembly) code is generated during the process, but is discarded after the compilation
* you first have to know how to let your compiler emit and leave assembly code 
* how to do that exactly is language- or compiler-dependent


## 2-1. Go
### 2-1-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile go/add123.go`) into the specified file `go/add123.go`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `Go` kernel

In [ ]:
%%writefile go/add123.go

package pl06
func Add123(x int64) int64 {
    return x + 123;
}

### 2-1-2. How to emit assembly code
* ordinary go compiler does not support emitting native 64 bit x86 assembly
* it instead emits something called "plan 9 assembly" ([details](https://go.dev/doc/asm)), which is a machine-independent, therefore portable, assembly
* portability is good, but for the learning objective of this exercise, it obscures how your program executes on the real machine
* therefore, in this exercise, we instead use an alternative compiler, `gccgo`, which generates a native assembly for x86
  * `-S` to emit assembly code
  * `-O` to optimize
  * `-g0` not to generate debugging information (unncessary for now)

In [ ]:
gccgo -O -g0 -S go/add123.go -o go/add123.s
cat go/add123.s


* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `Add123` (it may not be exactly the same)

* **NOTE 1:**
  * you need to say `package xxx`, with _xxx_ anything other than `main`; if you say `package main`, the compiler complains about the lack of `main` function
  * you need to <font color="red">capitalize the function name</font> whose assembly code you want to see, as otherwise the compiler may eliminate it as a dead code
  * in Go, functions starting with an uppercase letter are visible (can be called) from another package (see https://go.dev/doc/tutorial/create-module), so the compiler should leave the assembly code even if it finds the function not called anywhere in the file it is compiling

* **NOTE 2:** if you want to use `gccgo` along with `go build`, you can do
```
go build -compiler gccgo 
```
instead of just 
```
go build
```

## 2-2. Julia
### 2-2-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile jl/add123.jl`) into the specified file `jl/add123.jl`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `Julia` kernel


In [ ]:
%%writefile jl/add123.jl

function add123(n :: Int64)
    n + 123
end

using InteractiveUtils
code_native(add123)


### 2-2-2. How to emit assembly code
* Julia has <font color="blue">code_native</font> function that prints assembly code of a function given particular concrete types for its input parameters
* running this program defines `add123` and then calls `code_native` function, which shows assembly code for a specified function

In [ ]:
julia jl/add123.jl

* you can alternatively write only the definition of a function in the `.jl` file and call `code_native` in the command line

In [ ]:
%%writefile jl/add123.jl

function add123(n :: Int64)
    n + 123
end

In [ ]:
julia --load jl/add123.jl --eval 'using InteractiveUtils; code_native(add123)'

* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `add123` (it may not be exactly the same)


* note that the function `add123` declares the type of the input parameter (`n`) as `Int64`; you could give a different type or omit its type altogether and see how the generated coce change
* you can alternatively specify input type as the second parameter of `code_native`, like `code_native(add123, (Int64,))` or `code_native(add123, (Float64,))` in which case you perhaps want to omit the type in the definition


## 2-3. OCaml
### 2-3-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile ml/add123.ml`) into the specified file `ml/add123.ml`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `OCaml` kernel

In [ ]:
%%writefile ml/add123.ml

let add123 n = n + 123
;;


### 2-3-2. How to emit assembly code
* `ocamlopt` is a "native compiler" that translates OCaml source into machine code (`ocamlc` compiles OCaml to bytecode)
  * `-S` to emit assembly code


In [ ]:
ocamlopt -S ml/add123.ml
cat ml/add123.s

* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `add123` (it may not be exactly the same)

## 2-4. Rust
### 2-4-1. A simple example
* when executed by SHIFT + ENTER, the following cell simply writes the content (excluding the first `%%writefile rs/add123.rs`) into the specified file `rs/add123.rs`
* note that it is a feature of Python kernel and you don't have to (and should not) change it to `Rust` kernel


In [ ]:
%%writefile rs/add123.rs

#[no_mangle]
pub fn add123(n : i64) -> i64 {
    n + 123
}


### 2-4-2. How to emit assembly code
* `rustc` is a compiler for Rust, which is internally called by `cargo`
  * `-O` to optimize
  * `--emit asm` to generate assembly code
  * `--crate-type lib` to say this is for a library, not an executable

In [ ]:
rustc -O --emit asm --crate-type lib rs/add123.rs -o rs/add123.s
cat rs/add123.s

* try to locate the instruction that does `x + 123`
* try to locate the start of the function, which should be labeled with a symbol similar to the function name `add123` (it may not be exactly the same)

* **NOTE:**
  * `--crate-type lib` says you are building a library which will be called from another program, not an executable; without it, the compiler complains about the lack of `main` function
  * `#[no_mangle]` guarantees the label name corresponding to `add123` is exactly `add123` and has a side-effect of preventing the compiler from eliminating the function as dead code (I try to find a way to guarantee `add123` is not eliminatd without `#[no_mangle]`, but couldn't.  let me know if you know a better way)


# <font color="green"> Problem 1 :  Calling convention (where are args and return values?)</font>
* define a function `many_args` (or `Many_args` in Go) that takes many (> 10) integer parameters and returns the sum of them (similar to the following C function) in your language, compile it into assembly and examine it

In [ ]:
%%writefile cc/many_args.c

long many_args(long a00, long a01, long a02, long a03, long a04, long a05,
               long a06, long a07, long a08, long a09, long a10, long a11) {
  return a00 + a01 + a02 + a03 + a04 + a05 + a06 + a07 + a08 + a09 + a10 + a11;
}

In [ ]:
gcc -O3 -S cc/many_args.c -o cc/many_args.s
cat cc/many_args.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/many_args.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
gccgo -O -g0 -S go/many_args.go -o go/many_args.s
cat go/many_args.s

Julia

In [ ]:
julia --load jl/many_args.jl --eval 'using InteractiveUtils; code_native(many_args)'

OCaml

In [ ]:
ocamlopt -S ml/many_args.ml
cat ml/many_args.s

Rust

In [ ]:
rustc -O --emit asm --crate-type lib rs/many_args.rs -o rs/many_args.s
cat rs/many_args.s

* __<font color="blue">Questions:</font>__
  1. how are parameters passed in your language? which registers? memory?
  1. how is the return value passed in your language? which registers? memory?
  1. how does your language represent an integer $x$? does it use the _natural_ machine representation?



# <font color="green"> Problem 2 :  How floating point numbers are represented</font>
* define a function `add_floats` (or `Add_floats` in Go) that takes two floating point numbers and returns the sum of the two (similar to the following C function) in your language, compile it into assembly and examine it

In [ ]:
%%writefile cc/add_floats.c

double add_floats(double x, double y) {
  return x + y;
}

In [ ]:
gcc -O3 -S cc/add_floats.c -o cc/add_floats.s
cat cc/add_floats.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/add_floats.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
%%writefile go/add_floats.go

package pl06
func Add_floats(x float64, y float64) float64 {
    return x + y
}

In [ ]:
gccgo -O -g0 -S go/add_floats.go -o go/add_floats.s
cat go/add_floats.s

Julia

In [ ]:
%%writefile jl/add_floats.jl

function add_floats(x :: Float64, y :: Float64)
    x + y
end

In [ ]:
julia --load jl/add_floats.jl --eval 'using InteractiveUtils; code_native(add_floats)'

OCaml

In [ ]:
%%writefile ml/add_floats.ml

let add_floats x y = x +. y

In [ ]:
ocamlopt -S ml/add_floats.ml
cat ml/add_floats.s

Rust

In [ ]:
%%writefile rs/add_floats.rs

#[no_mangle]
pub fn add_floats(x : f64, y : f64) -> f64 {
    x + y
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/add_floats.rs -o rs/add_floats.s
cat rs/add_floats.s

* __<font color="blue">Questions:</font>__
  1. how are the two parameters passed in your language? which registers? memory?
  1. how does your language represent a floating point number $x$?
  1. is there any language whose floating point number computations will be significantly slower?



# <font color="green"> Problem 3 :  How structs are represented</font>
* define a struct `point` (or `Point`) that has two fields (`x` and `y`; both are 64 bit floating point numbers) and function `get_point_y` (or `Get_point_y`) that takes a point or a pointer to it and returns its `y` field

In [ ]:
%%writefile cc/get_struct_elem.c

typedef struct {
  double x;
  double y;
} point;
double get_point_y(point p) {
  return p.y;
}
double get_pointp_y(point * p) {
  return p->y;
}

In [ ]:
gcc -O3 -S cc/get_struct_elem.c -o cc/get_struct_elem.s
cat cc/get_struct_elem.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/get_struct_elem.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
%%writefile go/get_struct_elem.go

package pl06
type Point struct {
    x float64
    y float64
}
func Get_point_y(p Point) float64 {
    return p.y
}
func Get_pointp_y(p * Point) float64 {
    return p.y
}

In [ ]:
gccgo -O -g0 -S go/get_struct_elem.go -o go/get_struct_elem.s
cat go/get_struct_elem.s

Julia

In [ ]:
%%writefile jl/get_struct_elem.jl

struct Point
    x :: Float64
    y :: Float64
end
function get_point_y(p :: Point)
    p.y
end

In [ ]:
julia --load jl/get_struct_elem.jl --eval 'using InteractiveUtils; code_native(get_point_y)'

OCaml

In [ ]:
%%writefile ml/get_struct_elem.ml

type point_t = Point of float * float
;;
let get_point_y (Point(x, y)) = y
;;

In [ ]:
ocamlopt -S ml/get_struct_elem.ml
cat ml/get_struct_elem.s

Rust

In [ ]:
%%writefile rs/get_struct_elem.rs

pub struct Point {
    x : f64,
    y : f64
}
#[no_mangle]
pub fn get_point_y(p : Point) -> f64 {
    p.y
}
#[no_mangle]
pub fn get_pointp_y(p : &Point) -> f64 {
    return p.y;
}
#[no_mangle]
pub fn get_pointb_y(p : Box::<Point>) -> f64 {
    return p.y;
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/get_struct_elem.rs -o rs/get_struct_elem.s
cat rs/get_struct_elem.s

* __<font color="blue">Questions:</font>__
  1. how does your language represent `point` (or `Point`)?
  1. how does Go represent `*Point`?
  1. how does Rust represent `&Point` and `Box::<Point>`?



# <font color="green"> Problem 4 :  How arrays are represented</font>
* define a function `get_float_array_elem` (or `get_float_array_elem` in Go) that takes an array of floating point numbers, `a`, and an integer, `i`, and returns `i`-th element of `a` (similar to to the following C function) in your language, compile it into assembly and examine it
* for an array, use
  * `[]float64` (slice) for Go
  * `Vector{Float64}` for Julia
  * `float array` (slice) for OCaml
  * `Vec<f64>` for Rust

In [ ]:
%%writefile cc/get_float_array_elem.c

double get_float_array_elem_const(double a[10]) {
  return a[2];
}
double get_float_array_elem_i(double a[10], long i) {
  return a[i];
}

In [ ]:
gcc -O3 -S cc/get_float_array_elem.c -o cc/get_float_array_elem.s
cat cc/get_float_array_elem.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/get_float_array_elem.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
%%writefile go/get_float_array_elem.go

package pl06
func Get_float_array_elem_const(a []float64) float64 {
    return a[2]
}
func Get_float_array_elem_i(a []float64, i int64) float64 {
    return a[i]
}

In [ ]:
gccgo -O -g0 -S go/get_float_array_elem.go -o go/get_float_array_elem.s
cat go/get_float_array_elem.s

Julia

In [ ]:
%%writefile jl/get_float_array_elem.jl

function get_float_array_elem_const(a :: Vector{Float64})
    a[2]
end
function get_float_array_elem_i(a :: Vector{Float64}, i :: Int64)
    a[i]
end

In [ ]:
julia --load jl/get_float_array_elem.jl --eval 'using InteractiveUtils; code_native(get_float_array_elem_const)'

In [ ]:
julia --load jl/get_float_array_elem.jl --eval 'using InteractiveUtils; code_native(get_float_array_elem_i)'

OCaml

In [ ]:
%%writefile ml/get_float_array_elem.ml

let get_float_array_elem_const a = a.(2)
;;
let get_float_array_elem_i a i = a.(i)

In [ ]:
ocamlopt -S ml/get_float_array_elem.ml
cat ml/get_float_array_elem.s

Rust

In [ ]:
%%writefile rs/get_float_array_elem.rs

#[no_mangle]
pub fn get_float_array_elem_const(a : &[f64; 10]) -> f64 {
    a[2]
}
#[no_mangle]
pub fn get_float_array_elem_i(a : &[f64; 10], i : usize) -> f64 {
    a[i]
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/get_float_array_elem.rs -o rs/get_float_array_elem.s
cat rs/get_float_array_elem.s

* __<font color="blue">Questions:</font>__
  1. locate the instruction that gets the element from memory
  1. how does your language represent the respective array-like data structure?



# <font color="green"> Problem 5 :  If statement/expressions</font>
* define a function `collatz` (or `Collatz` in Go) that takes an integer $n$ and returns $n/2$ if $n$ is an even number and $3 n + 1$ otherwise (similar to the following C function)

In [ ]:
%%writefile cc/collatz.c

long collatz(long n) {
  if (n % 2 == 0) {
    return n / 2;
  } else {
    return 3 * n + 1;
  }
}

In [ ]:
gcc -O3 -S cc/collatz.c -o cc/collatz.s
cat cc/collatz.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/collatz.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
%%writefile go/collatz.go

package pl06
func Collatz(n int64) int64 {
    if n % 2 == 0 {
        return n / 2
    } else {
        return 3 * n + 1
    }
}

In [ ]:
gccgo -O -g0 -S go/collatz.go -o go/collatz.s
cat go/collatz.s

Julia

In [ ]:
%%writefile jl/collatz.jl

function collatz(n :: Int64)
  if n % 2 == 0
      div(n, 2)
  else
      3 * n + 1
  end
end

In [ ]:
julia --load jl/collatz.jl --eval 'using InteractiveUtils; code_native(collatz)'

OCaml

In [ ]:
%%writefile ml/collatz.ml

let collatz n =
  if n mod 2 = 0 then
    n / 2
  else
    3 * n + 1
;;

In [ ]:
ocamlopt -S ml/collatz.ml
cat ml/collatz.s

Rust

In [ ]:
%%writefile rs/collatz.rs

#[no_mangle]
pub fn collatz(n : i64) -> i64 {
    if n % 2 == 0 {
        n / 2
    } else {
        3 * n + 1
    }
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/collatz.rs -o rs/collatz.s
cat rs/collatz.s

* __<font color="blue">Questions:</font>__
  1. locate the conditional jump instruction that jumps depending on the result of the comparison



# <font color="green"> Problem 6 :  loops</font>
* define a function `sum_array_loop` (or `sum_array_loop` in Go) that takes an array of floating point numbers and returns the sum of all its elements (similar to to the following C function) in your language, compile it into assembly and examine it
* OCaml does have a loop, but you don't have to do it in OCaml
* use
  * `[]float64` (slice) in Go
  * `Vector{Float64}` in Julia
  * `Vec::<f64>` in Rust
for an array


In [ ]:
%%writefile cc/sum_array_loop.c

double sum_array_loop(double * a, long n) {
  double s = 0.0;
  for (int i = 0; i < n; i++) {
    s += a[i];
  }
  return s;
}

In [ ]:
gcc -O3 -S cc/sum_array_loop.c -o cc/sum_array_loop.s
cat cc/sum_array_loop.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/sum_array_loop.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
%%writefile go/sum_array_loop.go

package pl06
func Sum_array_loop(a []float64, n int64) float64 {
    s := 0.0
    var i int64;
    for i = 0; i < n; i++ {
        s += a[i]
    }
    return s;
}

In [ ]:
gccgo -O -g0 -S go/sum_array_loop.go -o go/sum_array_loop.s
cat go/sum_array_loop.s

Julia

In [ ]:
%%writefile jl/sum_array_loop.jl

function sum_array_loop(a :: Vector{Float64}, n :: Int64)
    s = 0.0
    for i = 1:n
        s += a[i];
    end
    return s
end

In [ ]:
julia --load jl/sum_array_loop.jl --eval 'using InteractiveUtils; code_native(sum_array_loop_loop)'

OCaml

In [ ]:
%%writefile ml/sum_array_loop.ml


In [ ]:
ocamlopt -S ml/sum_array_loop.ml
cat ml/sum_array_loop.s

Rust

In [ ]:
%%writefile rs/sum_array_loop.rs

#[no_mangle]
pub fn sum_array_floats_loop(a : &[f64], n : usize) -> f64 {
    let mut s = 0.0;
    for i in 0..n {
        s += a[i];
    }
    s
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/sum_array_loop.rs -o rs/sum_array_loop.s
cat rs/sum_array_loop.s

* __<font color="blue">Questions:</font>__
  * given that the array has a large number of elements, identify the loop that takes most of the time
  * in that loop, how many instructions are executed per array element?
  * will there be a performance difference between the loop version and tail recursive call version?
  * what does each language/compiler do to optimize it?



# <font color="green"> Problem 7 :  Function call</font>
* define a function `sum_array_rec` (or `sum_array_rec` in Go) that takes an array $a$ and two integers $p$ and $q$ and returns sum of $a[p:q]$
* write it in the divide and conquer recursion (similar to the following C function)


In [ ]:
%%writefile cc/sum_array_rec.c

double sum_array_rec(double * a, long p, long q) {
  if (q - p == 0) {
    return 0.0;
  } else if (q - p == 1) {
    return a[p];
  } else {
    long r = (p + q) / 2;
    return sum_array_rec(a, p, r) + sum_array_rec(a, r, q);
  }
}

In [ ]:
gcc -O -S cc/sum_array_rec.c -o cc/sum_array_rec.s
cat cc/sum_array_rec.s

* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/sum_array_rec.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
%%writefile go/sum_array_rec.go

package pl06
func Sum_array_rec(a []float64, p int64, q int64) float64 {
    if (q - p == 0) {
        return 0.0
    } else if (q - p == 1) {
        return a[p]
    } else {
        r := (p + q) / 2
        return Sum_array_rec(a, p, r) + Sum_array_rec(a, r, q)
    }
}

In [ ]:
gccgo -O -g0 -S go/sum_array_rec.go -o go/sum_array_rec.s
cat go/sum_array_rec.s

Julia

In [ ]:
%%writefile jl/sum_array_rec.jl

function sum_array_rec(a :: Vector{Float64}, p :: Int64, q :: Int64)
    if q - p == 0
        0.0
    elseif q - p == 1
        a[p]
    else
        r = (p + q) / 2
        sum_array_rec(a, p, r) + sum_array_rec(a, r, q)
    end
end

In [ ]:
julia --load jl/sum_array_rec.jl --eval 'using InteractiveUtils; code_native(sum_array_rec)'

OCaml

In [ ]:
%%writefile ml/sum_array_rec.ml

let rec sum_array_rec a p q = 
  if q - p = 0 then
    0.0
  else if q - p = 1 then
    a.(p)
  else 
    let r = (p + q) / 2 in
    (sum_array_rec a p r) +. (sum_array_rec a r q)
;;

In [ ]:
ocamlopt -S ml/sum_array_rec.ml
cat ml/sum_array_rec.s

Rust

In [ ]:
%%writefile rs/sum_array_rec.rs

pub fn sum_array_rec(a : &[f64], p : usize, q : usize) -> f64 {
    if q - p == 0 {
        0.0
    } else if q - p == 1 {
        a[p]
    } else {
        let r = (p + q) / 2;
        sum_array_rec(a, p, r) + sum_array_rec(a, r, q)
    }
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/sum_array_rec.rs -o rs/sum_array_rec.s
cat rs/sum_array_rec.s

* __<font color="blue">Questions:</font>__
  1. locate the instruction that recursively calls `Sum_Array_Rec`
  1. as the function computes `sum_array_rec(n - 1) + n - 1`, the value of `n` (or `n - 1`) must be saved somewhere during `sum_array_rec(n - 1)`; where is it saved?



# <font color="green"> Problem 8 :  Tail recursive call</font>
* define a function `sum_array_tail_rec` (or `Sum_array_tail_rec` in Go) that takes an array $a$, two integers $i$ and $n$, and a floating point number $s$, and returns $s + a[i:n]$
* write it in the _tail recursive_ manner (similar to the following C function)


In [ ]:
%%writefile cc/sum_array_tail_rec.c

double sum_array_tail_rec(double * a, long i, long n, double s) {
  if (i == n) {
    return s;
  } else {
    return sum_array_tail_rec(a, i + 1, n, s + a[i]);
  }
}

In [ ]:
gcc -O2 -S cc/sum_array_tail_rec.c -o cc/sum_array_tail_rec.s
cat cc/sum_array_tail_rec.s


* generally, a function call whose return value becomes the return value of the caller is called a _tail call_
* for example, the function call `g(x)` below is a tail call, but `h(x)` is not
```
def f(x):
   if ...:
      return g(x)
   else:
      return h(x) + 1
```
* a _tail call_ that is also a recursive call is called a _tail recursive call_ and a compiler generally has a chance to optimize tail recursive calls
* a loop can generally be expressed as tail recursive functions
```
while A:
  B
```
can be translated into something like
```
def loop(...):
  if A:
     B
     loop(...)
```
* in particular, in OCaml, where loop syntax exist but more functional way of writing things is encouraged, writing what would a loop in other languages in tail recursive functions is a trick you want to master


* write the code below
* <font color="red">replace xx with go, jl, ml, or rs and write the function</font>

In [ ]:
%%writefile xx/sum_array_tail_rec.xx


* and see the assembly code by executing one of the following

Go

In [ ]:
%%writefile go/sum_array_tail_rec.go

package pl06
func Sum_array_tail_rec(a []float64, i int64, n int64, s float64) float64 {
    if i == n {
        return s
    } else {
        return Sum_array_tail_rec(a, i + 1, n, s + a[i])
    }
}

In [ ]:
gccgo -O -g0 -S go/sum_array_tail_rec.go -o go/sum_array_tail_rec.s
cat go/sum_array_tail_rec.s

Julia

In [ ]:
%%writefile jl/sum_array_tail_rec.jl

function sum_array_tail_rec(a :: Vector{Float64}, i :: Int64, n :: Int64, s :: Float64)
    if i == n
        s
    else
        sum_array_tail_rec(a, i + 1, n, s + a[i])
    end
end

In [ ]:
julia --load jl/sum_array_tail_rec.jl --eval 'using InteractiveUtils; code_native(sum_array_tail_rec)'

OCaml

In [ ]:
%%writefile ml/sum_array_tail_rec.ml

let rec sum_array_tail_rec a i n s = 
  if i = n then
    s
  else 
    sum_array_tail_rec a (i + 1) n (s + a.(i))
;;

In [ ]:
ocamlopt -S ml/sum_array_tail_rec.ml
cat ml/sum_array_tail_rec.s

Rust

In [ ]:
%%writefile rs/sum_array_tail_rec.rs

#[no_mangle]
pub fn sum_array_floats_tail_rec(a : &[f64], i : usize, n : usize, s : f64) -> f64 {
    if i == n {
        s
    } else {
        sum_array_floats_tail_rec(a, i + 1, n, s + a[i])
    }
}

In [ ]:
rustc -O --emit asm --crate-type lib rs/sum_array_tail_rec.rs -o rs/sum_array_tail_rec.s
cat rs/sum_array_tail_rec.s

* __<font color="blue">Questions:</font>__
  1. does your language use `call` instruction or successfully eliminate it (compile it into a loop)?
